In [3]:
!pip install selenium

     |████████████████████████████████| 904 kB 1.5 MB/s eta 0:00:01


In [4]:
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver

In [7]:
driver = webdriver.Edge()
driver.get('https://www.avito.ru/moskva/nedvizhimost')
#button = driver.find_element_by_xpath('//button[@class="button button-origin button-origin_large button-origin_full-width recommendations-show-more js-recommendations-show-more"]').click()

WebDriverException: Message: 'MicrosoftWebDriver.exe' executable needs to be in PATH. Please download from http://go.microsoft.com/fwlink/?LinkId=619687


In [ ]:
driver.find_elements_by_xpath()